# Training a text classifier model with fastai
- this notebook assumes you have already run text_model_training.ipynb notebook
- In this notebook, the IMDB dataset is ingested
- the first section

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from fastai.text.all import *

In [3]:
modifier = 'mar3'

# Ingest the dataset
- define the path for the dataset
- create a TextDataLoaders object

In [4]:
%%time
# create dataloaders object
'''dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', bs=16)
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)'''
# LSTM have multiple dropout probabilities for different things. Once you set them, this drop_mult property scales all of them. So you can change all dropout probabilities simultaneously using this, keeping their relative size
path = untar_data(URLs.IMDB)
path.ls()

CPU times: user 0 ns, sys: 5.72 ms, total: 5.72 ms
Wall time: 11.1 ms


(#7) [Path('/storage/data/imdb/README'),Path('/storage/data/imdb/tmp_lm'),Path('/storage/data/imdb/imdb.vocab'),Path('/storage/data/imdb/tmp_clas'),Path('/storage/data/imdb/test'),Path('/storage/data/imdb/train'),Path('/storage/data/imdb/unsup')]

In [5]:
%%time
# dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', bs=8)
# get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls = TextDataLoaders.from_folder(path, valid = 'test', is_lm=True, bs=16)

CPU times: user 4.75 s, sys: 5.39 s, total: 10.1 s
Wall time: 37 s


In [6]:
dls.show_batch(max_n=4)

,text,text_
0,"xxbos 50 years old , this musical comedy fantasy might look its age , but it wears it with dignity . \n\n xxmaj this film is still great fun . xxmaj crosby was never really romantic lead material , but he delivers the material with the lightly humorous edge it needs . xxmaj bendix plays broad and is huge fun in a part which calls upon his strengths . xxmaj hardwicke -","50 years old , this musical comedy fantasy might look its age , but it wears it with dignity . \n\n xxmaj this film is still great fun . xxmaj crosby was never really romantic lead material , but he delivers the material with the lightly humorous edge it needs . xxmaj bendix plays broad and is huge fun in a part which calls upon his strengths . xxmaj hardwicke - how"
1,"something despite stating that he wants to escape so he can see his daughter again , and xxmaj kate becomes emotionally tough seconds after going to pieces over someone that ripped her off for a xxunk . xxmaj yeah . \n\n xxmaj after starting out as a "" this could happen to anyone "" movie , it quickly falls apart as it introduces ideas that make it more and more unrealistic .","despite stating that he wants to escape so he can see his daughter again , and xxmaj kate becomes emotionally tough seconds after going to pieces over someone that ripped her off for a xxunk . xxmaj yeah . \n\n xxmaj after starting out as a "" this could happen to anyone "" movie , it quickly falls apart as it introduces ideas that make it more and more unrealistic . a"
2,"--polarisdib xxbos xxmaj this film takes you on one family 's impossible journey , and makes you feel every step of their odyssey . xxmaj beautifully acted and photographed , heartbreakingly real . xxmaj its last line , with its wistful hope , is one of the more powerful in memory . xxbos xxup ok , this has got 2 be one of the worst excuses 4 a movie that i have","xxbos xxmaj this film takes you on one family 's impossible journey , and makes you feel every step of their odyssey . xxmaj beautifully acted and photographed , heartbreakingly real . xxmaj its last line , with its wistful hope , is one of the more powerful in memory . xxbos xxup ok , this has got 2 be one of the worst excuses 4 a movie that i have ever"
3,"xxmaj really bad and redundant special effects , zombies that look like normal people ( except for a white additional skin pulled over their faces ) , xxup way xxup to xxup much fake blood ( i like realism a lot , the combination of realism and xxmaj zombie films being debatable , but the presented gore is just plain silly ) . xxmaj the camera stays quite long with xxunk scenes","really bad and redundant special effects , zombies that look like normal people ( except for a white additional skin pulled over their faces ) , xxup way xxup to xxup much fake blood ( i like realism a lot , the combination of realism and xxmaj zombie films being debatable , but the presented gore is just plain silly ) . xxmaj the camera stays quite long with xxunk scenes ,"


# Define classifier

In [23]:
# dls definition cribbed from chapter 10
# this works
'''
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)



The independent variable is often referred to as x, and the dependent variable is often referred to as y.
Here, we are telling fastai what function to call to create the labels in our dataset: 
get_y=parent_label parent_label is a function provided by fastai that simply gets the name of the 
folder a file is in. Because we put each of our bear images into folders based on the type of bear, 
this is going to give us the labels that we need.

'''

In [11]:
# refactored definition of dataloader object
'''dls = TextDataLoaders.from_df(
    df_tok, path=path, 
    vocab = make_vocab(count),
    text_col = 'text',label_col='label')
'''

dls_clas = TextDataLoaders.from_folder(path,
    blocks=(TextBlock.from_folder(path, vocab=dls.vocab),CategoryBlock),
#path, vocab=dls.vocab,     
#    text_col= 'text', 
#   label_col='label',
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test'),
#    get_items=partial(get_text_files, folders=['train', 'test']),
#    splitter=GrandparentSplitter(valid_name='test'), 
    bs=128, seq_len=72)

In [12]:
path

Path('/storage/data/imdb')

In [13]:
%%time
# define a text_classifier_learner object
learn_clas = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

CPU times: user 6.35 s, sys: 1.25 s, total: 7.6 s
Wall time: 4.99 s


In [14]:
%%time
# set the path to the location of the encoder
learn_clas.path = Path('/notebooks/temp')

CPU times: user 25 µs, sys: 7 µs, total: 32 µs
Wall time: 37.2 µs


In [15]:
# load the encoder that was saved when the language model was trained
learn_clas = learn_clas.load_encoder('ft_'+modifier)

In [16]:
%%time
learn_clas.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.412429,None,None,01:39


/opt/conda/envs/fastai/lib/python3.8/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


CPU times: user 1min 7s, sys: 29.8 s, total: 1min 37s
Wall time: 1min 39s


In [17]:
%%time
learn_clas.freeze_to(-2)
learn_clas.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.250098,None,None,02:02


/opt/conda/envs/fastai/lib/python3.8/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


CPU times: user 1min 24s, sys: 36.8 s, total: 2min
Wall time: 2min 2s


In [18]:
preds = learn_clas.predict("this film shows incredibly bad writing and is a complete disaster")

In [19]:
preds

('neg', TensorText(0), TensorText([9.9997e-01, 2.6275e-05]))

In [22]:
preds = learn_clas.predict("what a terrible film")

In [23]:
preds

('neg', TensorText(0), TensorText([9.9956e-01, 4.4422e-04]))

In [17]:
learn_clas.save('classifier_single_epoch_'+modifier+'b')

Path('/notebooks/temp/models/classifier_single_epoch_mar3b.pth')